In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
year_start = 2010
mounth_start = 1

year_stop = 2017
mounth_stop = 12

data_dict = {"title" : [] ,
"link" : [] ,
"date" : [],
"author" : [],
"text" : [],
"comment_rbloggers" : [] 
}

years = []
mounths = []
for year in range(year_start, year_stop + 1):
    years.append(year)
    mounths_year = []
    if year_stop == year_start :
        for mounth in range(mounth_start, mounth_stop + 1):
            mounths_year.append(mounth)
    else :
        if year == year_stop :
            for mounth in range(1, mounth_stop + 1):
                mounths_year.append(mounth)
        else:
            for mounth in range(mounth_start, 12 + 1):
                mounths_year.append(mounth)
    mounths.append(mounths_year)

try :
    for year, mounths_year in zip(years , mounths):
        for mounth in mounths_year :
            print(year, " ", mounth)
            source_n = requests.get(f"https://www.r-bloggers.com/{year}/{mounth}/").text 
            soup_n = BeautifulSoup(source_n, 'lxml')
            pages_idx = soup_n.find_all("a" ,class_ ="page-numbers")
            if len(pages_idx) == 0:
                break
            last_page_n = int(pages_idx[-2].text)

            for num_page in range(1, last_page_n + 1):

                source = requests.get(f"https://www.r-bloggers.com/{year}/{mounth}/page/{num_page}/").text 
                soup = BeautifulSoup(source, 'lxml')
                articles = soup.find_all("article")
                for idx_article in range(len(articles)):
                    article = articles[idx_article]
                    data_dict["author"].append(article.find("span", class_ = "vcard author").text)
                    data_dict["title"].append(article.find("h3", class_ = "loop-title").text)
                    data_dict["date"].append(article.find("p", class_ = "meta").text.split(" |")[0])
                    link = str(article.find("h3", class_ = "loop-title").find("a")).split("href=")[1].split(' rel')[0][1:-1]
                    data_dict["link"].append(link)
                    source_link = requests.get(link).text 
                    soup_link = BeautifulSoup(source_link, 'lxml')
                    text = ""
                    for i, para in enumerate(soup_link.find_all("p")):
                        if i == 0 :
                            try :
                                data_dict["comment_rbloggers"].append(para.get_text().split("|")[1])
                            except :
                                data_dict["comment_rbloggers"].append(np.NaN)
                        if i in (0,1, len(soup_link.find_all("p")) - 1) :
                            pass
                        else:
                            text = text + " " + para.get_text()
                    data_dict["text"].append(text[1:])
                    
        df = pd.DataFrame(data_dict, columns = data_dict.keys())
        df.to_csv("data_rbloggers_{year}.csv", sep = "\t", index = False)

    df = pd.DataFrame(data_dict, columns = data_dict.keys())
    df.to_csv("data_rbloggers_2010-2017.csv", sep = "\t", index = False)
except Exception as e:
    print(e)
    df = pd.DataFrame(data_dict, columns = data_dict.keys())
    df.to_csv("data_rbloggers_2010-2017.csv", sep = "\t", index = False)

2010   1
2010   2
2010   3
2010   4
2010   5
2010   6
2010   7
2010   8
2010   9
2010   10
2010   11
2010   12
2011   1
2011   2
2011   3
2011   4
2011   5
2011   6
2011   7
2011   8
2011   9
2011   10
2011   11
2011   12
2012   1
2012   2
2012   3
2012   4
2012   5


In [ ]:
print("ok")

In [355]:
df.to_csv("data_rbloggers_final.csv", sep = "\t", index = False)

In [314]:
df = pd.read_csv("data_rbloggers.csv", sep = "\t")

In [334]:
df.tail()

,title,link,date,author,text,comment_rbloggers
11639,Convolutional Neural Networks,https://www.r-bloggers.com/2021/10/convolution...,"October 1, 2021",finnstats,"Convolutional neural networks, Let’s look at a...",0 Comments
11640,Using Java functions from JAR file in R using ...,https://www.r-bloggers.com/2021/10/using-java-...,"October 1, 2021",sang-heon lee,NaN,0 Comments
11641,September 2021 ISC Call for Proposals – Now Open!,https://www.r-bloggers.com/2021/10/september-2...,"October 1, 2021",R Consortium,The deadline for submitting proposals is Octob...,0 Comments
11642,Crash Course in R Model Deployment with Docker...,https://www.r-bloggers.com/2021/10/crash-cours...,"October 1, 2021",Gary Hutson,I have put together a complete guide to model ...,0 Comments
11643,OASIS UI technical updates,https://www.r-bloggers.com/2021/10/oasis-ui-te...,"October 1, 2021",Mirai Solutions,Mirai Solutions stays engaged in the developme...,0 Comments


In [332]:
df.link[0]

'https://www.r-bloggers.com/2018/01/easily-converting-strings-to-times-and-dates-in-r-with-fliptime/'

In [333]:
df.title[0]

'Easily Converting Strings to Times and Dates in R with flipTime'

In [320]:
df.author.value_counts()

rOpenSci - open tools for open science    250
Thinking inside the box                   234
R Views                                   222
John Mount                                210
xi'an                                     192
                                         ... 
Jason Hoffmeier                             1
Lou Bajuk, Carl Howe                        1
Piotr Fic                                   1
Adam Gabriel Dobrakowski                    1
Verena                                      1
Name: author, Length: 1256, dtype: int64

In [324]:
df.title[10]

'Analysis of Trump’s State of the Union Speech, with R'

In [349]:
df.text[7]

'Hi all, so given our logo here at Jumping Rivers is a set of lines designed to look like a Gaussian Process, we thought it would be a neat idea to recreate this image in R. To do so we’re going to need a couple packages. We do the usual install.packages() dance (remember this step can be performed in parallel) We’re also going to need the data containing the points for the lines and which set of points belongs to which line. There is a Gist available to download via Jumping Rivers. To read in the csv file we’re going to use the raw data link. The data set contains three columns, x, y and type, where type indicates the line. Let’s start with a standard geom_line()  The graph shares similarities with our logo, but is too discrete. To smooth the curve, we’ll use a function from the ggalt package  The function geom_xspline() is the X-spline version of geom_line(), drawing a curve relative to the observations.\nThe parameterspline_shape = -0.3 controls the shape of the spline relative to t

In [352]:
df['text'] = df.text.apply(lambda x : str(x).replace("\n", " "))

In [354]:
df.head()

,title,link,date,author,text,comment_rbloggers
0,Easily Converting Strings to Times and Dates i...,https://www.r-bloggers.com/2018/01/easily-conv...,"January 31, 2018",Mathew McLean,Date conversion in R can be a real pain. Howev...,0 Comments
1,The importance of context,https://www.r-bloggers.com/2018/01/the-importa...,"January 31, 2018",Dr. Michael Green,When we do modeling it’s of utmost importance ...,0 Comments
2,PCA in a tidy(verse) framework,https://www.r-bloggers.com/2018/01/pca-in-a-ti...,"January 31, 2018",Rstats on goonR blog,"The other day, a question was posted on RStudi...",0 Comments
3,The importance of context,https://www.r-bloggers.com/2018/01/the-importa...,"January 31, 2018",Dr. Michael Green,When we do modeling it’s of utmost importance ...,0 Comments
4,Our Logo In R,https://www.r-bloggers.com/2018/01/our-logo-in...,"January 31, 2018",R on The Jumping Rivers Blog,"Hi all, so given our logo here at Jumping Rive...",0 Comments
